In [1]:
import pandas as pd
import math
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
from datetime import datetime, timedelta, timezone
import pyodbc
from sqlalchemy import Column, Date, Integer, String, Numeric, create_engine, Float, inspect, func, MetaData, Table, select, ForeignKey, LargeBinary
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import mapper, registry, Session, relationship
from sqlalchemy.ext.automap import automap_base
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, mean_squared_error, balanced_accuracy_score 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
import scipy
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [7]:
#SQLAlchemy Setup
def get_engine():
    SERVER = 'localhost'
    DATABASE = 'metmast'
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect=DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes;')
    return engine

engine = get_engine()
metadata = MetaData()
metadata.reflect(bind=engine)
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)
mapper_registry = registry()

In [4]:
#########################################################################################################################################################################################
#data_meta
class data_meta:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True)
            ]

# Create table
metadata = MetaData()
data_meta_table= Table('data_meta', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = data_meta,
    local_table = data_meta_table
)
#########################################################################################################################################################################################
#feature
class feature:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("data_meta_id", Integer, ForeignKey(data_meta.id)),
            Column("name", String),
            Column("type_", String),
            Column("description", String)
            ]

# Create table
metadata = MetaData()
feature_table= Table('feature', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = feature,
    local_table = feature_table
)
#########################################################################################################################################################################################
#label
class label:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("data_meta_id", Integer, ForeignKey(data_meta.id)),
            Column("name", String),
            Column("description", String)
            ]

# Create table
metadata = MetaData()
label_table= Table('label', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = label,
    local_table = label_table
)
#########################################################################################################################################################################################
#label_categorical
class label_categorical:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("label_id", Integer, ForeignKey(data_meta.id)),
            Column("category", String),
            Column("description", String)
            ]

# Create table
metadata = MetaData()
label_categorical_table= Table('label_categorical', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = label_categorical,
    local_table = label_categorical_table
)
#########################################################################################################################################################################################
#datapoint_mappings
class datapoint_mappings:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("data_meta_id", Integer, ForeignKey(data_meta.id)),
            Column("grouping", String)
            ]

# Create table
metadata = MetaData()
datapoint_mappings_table= Table('datapoint_mappings', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = datapoint_mappings,
    local_table = datapoint_mappings_table
)
#########################################################################################################################################################################################
#datapoint
class datapoint:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("datapoint_mappings_id", Integer, ForeignKey(datapoint_mappings.id))
            ]

# Create table
metadata = MetaData()
datapoint_table= Table('datapoint', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = datapoint,
    local_table = datapoint_table
)
#########################################################################################################################################################################################
#datapoint_feature_value_int
class datapoint_feature_value_int:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("datapoint_id", Integer, ForeignKey(datapoint.id)),
            Column("feature_id", Integer, ForeignKey(feature.id)),
            Column("int_", Integer)
            ]

# Create table
metadata = MetaData()
datapoint_feature_value_int_table= Table('datapoint_feature_value_int', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = datapoint_feature_value_int,
    local_table = datapoint_feature_value_int_table
)
#########################################################################################################################################################################################
#datapoint_feature_value_float
class datapoint_feature_value_float:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("datapoint_id", Integer, ForeignKey(datapoint.id)),
            Column("feature_id", Integer, ForeignKey(feature.id)),
            Column("float_", Float)
            ]

# Create table
metadata = MetaData()
datapoint_feature_value_float_table = Table('datapoint_feature_value_float', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = datapoint_feature_value_float,
    local_table = datapoint_feature_value_float_table
)
#########################################################################################################################################################################################
#datapoint_feature_value_string
class datapoint_feature_value_string:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True),
            Column("datapoint_id", Integer, ForeignKey(datapoint.id)),
            Column("feature_id", Integer, ForeignKey(feature.id)),
            Column("string_", String)
            ]

# Create table
metadata = MetaData()
datapoint_feature_value_string_table= Table('datapoint_feature_value_string', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = datapoint_feature_value_string,
    local_table = datapoint_feature_value_string_table
)

<Mapper at 0x2449170cc40; datapoint_feature_value_string>

In [9]:
###############################################################################################################  
#Create meta informations if not existent
class train_process:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("placeHolder", String)]

# Create table
metadata = MetaData()
train_process_table= Table('train_process', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = train_process,
    local_table = train_process_table
)
###############################################################################################################  
#Create meta informations if not existent
class datapoint_train_process_junction:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("datapoint_id", Integer, ForeignKey(datapoint.id)),
            Column("train_process_id", Integer, ForeignKey(train_process.id)),            
            ]

# Create table
metadata = MetaData()
datapoint_train_process_junction_table= Table('datapoint_train_process_junction', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = datapoint_train_process_junction,
    local_table = datapoint_train_process_junction_table
)
###############################################################################################################
#Create meta informations if not existent
class train_process_init_parameter:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("train_process_id", Integer, ForeignKey(train_process.id)),
            Column("min_threshold_feature_variance", Float),
            Column("max_threshold_feature_variance", Float)]

# Create table
metadata = MetaData()
train_process_init_parameter_table = Table('train_process_init_parameter', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = train_process_init_parameter,
    local_table = train_process_init_parameter_table
)
###############################################################################################################
#Create meta informations if not existent
class train_process_statistics:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("train_process_id", Integer, ForeignKey(train_process.id)),
            Column("elapsedTime", Integer)
            ]

# Create table
metadata = MetaData()
train_process_statistics_table = Table('train_process_statistics', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = train_process_statistics,
    local_table = train_process_statistics_table
)
###############################################################################################################
#Create meta informations if not existent
class train_process_iteration:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("train_process_id", Integer, ForeignKey(train_process.id))
            ]

# Create table
metadata = MetaData()
train_process_iteration_table = Table('train_process_iteration', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = train_process_iteration,
    local_table = train_process_iteration_table
)
###############################################################################################################
#Create meta informations if not existent
class hyperparameter:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("train_process_iteration_id", Integer, ForeignKey(train_process_iteration.id)),
            Column("threshold_feature_variance", Integer),
            Column("max_depth", Integer),
            Column("min_samples_leaf", Integer),
            Column("random_state", Integer),
            Column("max_features", Integer),
            Column("criterion", String),
]

# Create table
metadata = MetaData()
hyperparameter_table = Table('hyperparameter', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = hyperparameter,
    local_table = hyperparameter_table
)
###############################################################################################################
#Create meta informations if not existent
class train_process_iteration_score:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("train_process_iteration_id", Integer, ForeignKey(train_process_iteration.id)),
            Column("balanced_accuracy_score", Integer),            
            ]

# Create table
metadata = MetaData()
train_process_iteration_score_table = Table('train_process_iteration_score', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = train_process_iteration_score,
    local_table = train_process_iteration_score_table
)

###############################################################################################################
#Create meta informations if not existent
class train_process_iteration_compute_result:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("train_process_iteration_id", Integer, ForeignKey(train_process_iteration.id))          
            ]

# Create table
metadata = MetaData()
train_process_iteration_compute_result_table = Table('train_process_iteration_compute_result', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = train_process_iteration_compute_result,
    local_table = train_process_iteration_compute_result_table
)

###############################################################################################################
#Create meta informations if not existent
class dropped_feature_variance_filter:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("train_process_iteration_compute_result_id", Integer, ForeignKey(train_process_iteration_compute_result.id)), 
            Column("feature_id", Integer, ForeignKey(Base.classes["feature"].id)),  
            Column("feature_variance", Float)          
]

# Create table
metadata = MetaData()
dropped_feature_variance_filter_table = Table('dropped_feature_variance_filter', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = dropped_feature_variance_filter,
    local_table = dropped_feature_variance_filter_table
)
###############################################################################################################
#Create meta informations if not existent
class model:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("path_to_model", String),
            Column("model_binary", LargeBinary)
            ]

# Create table
metadata = MetaData()
model_table = Table('model', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = model,
    local_table = model_table
)
###############################################################################################################
#Create meta informations if not existent
class prediciions_categorical:
    pass

# Create columns
columns = [ Column('id', Integer, primary_key=True), 
            Column("datapoint_train_process_junction_id", Integer, ForeignKey(datapoint_train_process_junction.id)),
            Column("model_id", Integer, ForeignKey(model.id)),
            Column("pred", String)
            ]

# Create table
metadata = MetaData()
prediciions_categorical_table = Table('prediciions_categorical', metadata, *columns)
metadata.create_all(engine)

# Map the class imperatively
mapper_registry.map_imperatively(
    class_ = prediciions_categorical,
    local_table = prediciions_categorical_table
)


<ipython-input-9-a96084390b05>:16: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.train_process, and will be replaced in the string-lookup table.
  mapper_registry.map_imperatively(
<ipython-input-9-a96084390b05>:37: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.datapoint_train_process_junction, and will be replaced in the string-lookup table.
  mapper_registry.map_imperatively(
<ipython-input-9-a96084390b05>:58: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.train_process_init_parameter, and will be replaced in the string-lookup table.
  mapper_registry.map_imperatively(
<ipython-input-9-a96084390b05>:79: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.train_process_statistics, and will be replaced in the string-lookup table.
  mappe

<Mapper at 0x24493b379a0; model>